In [4]:
import os
import datetime
from datetime import datetime, timedelta

import numpy as np
import pandas as pd

import _pickle as cPickle
import gzip
import dask
from dask import compute, delayed
import functools
import warnings
warnings.filterwarnings('ignore')

In [5]:
# 根目录HEAD_PATH
HEAD_PATH = "order book data"
# 数据保存根目录
SAVE_PATH = "order book data"
# 样本的目录
DATA_PATH_1 = HEAD_PATH + "/order book tick/"
DATA_PATH_2 = HEAD_PATH + "/order flow tick/"
TMP_DATA_PATH = HEAD_PATH + "/tmp pkl/"
DEBUG_DATA_PATH = HEAD_PATH + "/tmp debug/"

In [6]:
def load(path):
    with gzip.open(path, 'rb', compresslevel=1) as file_object:
        raw_data = file_object.read()
    return cPickle.loads(raw_data)

def save(data, path):
    serialized = cPickle.dumps(data)
    with gzip.open(path, 'wb', compresslevel=1) as file_object:
        file_object.write(serialized)

def parLapply(CORE_NUM, iterable, func, *args, **kwargs):
    with dask.config.set(scheduler='processes', num_workers=CORE_NUM):
        f_par = functools.partial(func, *args, **kwargs)
        result = compute([delayed(f_par)(item) for item in iterable])[0]
        return result

In [9]:
CORE_NUM = int(os.environ['NUMBER_OF_PROCESSORS'])
product_list = ['cu', 'zn', 'ni', 'au', 'ag']
def construct_debug_data(date_pkl, product):
    data = load(TMP_DATA_PATH+product+'/'+date_pkl)
    data = data.iloc[:500, :]
    save(data, DEBUG_DATA_PATH+product+'/'+date_pkl)

for product in product_list:
    print(product)
    if not os.path.exists(DEBUG_DATA_PATH+product):
        os.makedirs(DEBUG_DATA_PATH+product)
    all_dates = np.array(os.listdir(TMP_DATA_PATH + product))
    parLapply(CORE_NUM, all_dates, construct_debug_data, product=product)

In [11]:
load(DEBUG_DATA_PATH+'cu/'+'20220104.pkl')

,TimeStamp,BidPrice1,BidVolume1,AskPrice1,AskVolume1,BidPrice2,BidVolume2,AskPrice2,AskVolume2,BidPrice3,...,BidPrice5,BidVolume5,AskPrice5,AskVolume5,weight_price,difference_of_price,m_neg,m_pos,ret,classify
TimeStamp,,,,,,,,,,,,,,,,,,,,,
2022-01-04 08:59:00.500,2022-01-04 08:59:00.500,70500.0,15,70550.0,46.0,70490.0,3,70570.0,104.0,70460.0,...,70440.0,5,70630.0,10.0,70512.295082,0.000000,70512.295082,70318.508702,-0.002748,-1
2022-01-04 09:00:00.500,2022-01-04 09:00:00.500,70460.0,17,70490.0,9.0,70450.0,12,70500.0,51.0,70440.0,...,70420.0,10,70540.0,18.0,70479.615385,-32.679697,70495.955233,70313.720451,-0.002585,-1
2022-01-04 09:00:01.000,2022-01-04 09:00:01.000,70450.0,2,70460.0,17.0,70440.0,7,70480.0,3.0,70430.0,...,70410.0,25,70510.0,8.0,70451.052632,-61.242450,70480.987699,70309.481247,-0.002433,-1
2022-01-04 09:00:01.500,2022-01-04 09:00:01.500,70430.0,1,70440.0,1.0,70420.0,10,70450.0,20.0,70410.0,...,70370.0,2,70490.0,34.0,70435.000000,-77.295082,70469.490775,70305.416828,-0.002328,-1
2022-01-04 09:00:02.000,2022-01-04 09:00:02.000,70400.0,7,70430.0,5.0,70390.0,1,70440.0,5.0,70370.0,...,70330.0,2,70480.0,3.0,70417.500000,-94.795082,70459.092620,70301.654008,-0.002234,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-04 09:04:07.500,2022-01-04 09:04:07.500,70090.0,6,70100.0,10.0,70080.0,34,70110.0,16.0,70070.0,...,70050.0,21,70140.0,10.0,70093.750000,17.750000,70081.851008,70057.589809,-0.000346,-1
2022-01-04 09:04:08.000,2022-01-04 09:04:08.000,70090.0,3,70100.0,11.0,70080.0,26,70110.0,16.0,70070.0,...,70050.0,21,70140.0,10.0,70092.142857,24.142857,70082.120056,70056.842124,-0.000361,-1
2022-01-04 09:04:08.500,2022-01-04 09:04:08.500,70080.0,26,70100.0,14.0,70070.0,23,70110.0,16.0,70060.0,...,70040.0,25,70140.0,10.0,70093.000000,15.000000,70082.536722,70056.134534,-0.000377,-1
